<img src= "http://www.voyagerconsulting.org/images/logo1.png">

# Voyager Consulting: Jira API

## Import Libraries

In [1]:
import requests
import json
import pandas as pd
import time
import datetime

In [2]:
conda install jira

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


## Configurations

In [3]:
from collections import Counter
from jira import JIRA
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [4]:
#Will connect to a Jira Instance started from the Atlassian Plugin SDK
jira = JIRA(basic_auth=("admin", "admin")) # a username/password tuple

# Get the mutable application properties for this server (requires
# jira-system-administrators permission)
props = jira.application_properties()

#Oauth
key_cert_data = None
with open(key_cert, 'r') as key_cert_file:
    key_cert_data = key_cert_file.read()

oauth_dict = {
    'access_token': 'foo',
    'access_token_secret': 'bar',
    'consumer_key': 'jira-oauth-consumer',
    'key_cert': key_cert_data
}
auth_jira = JIRA(oauth=oauth_dict)

{'response': None, 'request': <PreparedRequest [GET]>}\{'response': None, 'request': <PreparedRequest [GET]>}
{'response': None, 'request': <PreparedRequest [GET]>}\{'response': None, 'request': <PreparedRequest [GET]>}
{'response': None, 'request': <PreparedRequest [GET]>}\{'response': None, 'request': <PreparedRequest [GET]>}


ConnectionError: HTTPConnectionPool(host='localhost', port=2990): Max retries exceeded with url: /jira/rest/api/2/serverInfo (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fb4b12a6400>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [6]:
#Everything goes through the JIRA object
jira = JIRA('https://plutotv.atlassian.net')

## Issues

### Getting Issues

In [7]:
issue = jira.issue('JRA-1330')

#Get direct fields of issues
summary = issue.fields.summary         # 'Field level security permissions'
votes = issue.fields.votes.votes       # 440 (at least)

#Get specific fields
issue = jira.issue('JRA-1330', fields='summary,comment')

JIRAError: JiraError HTTP 404 url: https://plutotv.atlassian.net/rest/api/2/issue/JRA-1330
	text: Issue does not exist or you do not have permission to see it.
	
	response headers = {'Server': 'AtlassianProxy/1.15.8.1', 'Vary': 'Accept-Encoding', 'Cache-Control': 'no-cache, no-store, no-transform', 'Content-Type': 'application/json;charset=UTF-8', 'Content-Encoding': 'gzip', 'Strict-Transport-Security': 'max-age=315360000; includeSubDomains; preload', 'Date': 'Thu, 29 Oct 2020 17:08:12 GMT', 'ATL-TraceId': '5b8ce5dbbe156e25', 'X-AREQUESTID': '0b3d95b7-754b-4216-a0dc-3265a64122c3', 'X-XSS-Protection': '1; mode=block', 'Transfer-Encoding': 'chunked', 'Timing-Allow-Origin': '*', 'X-Content-Type-Options': 'nosniff', 'Connection': 'keep-alive', 'Expect-CT': 'report-uri="https://web-security-reports.services.atlassian.com/expect-ct-report/global-proxy", enforce, max-age=86400'}
	response text = {"errorMessages":["Issue does not exist or you do not have permission to see it."],"errors":{}}

### Searching Issues with JQL

In [8]:
# Search returns first 50 results, `maxResults` must be set to exceed this
issues_in_proj = jira.search_issues('project=PROJ')
all_proj_issues_but_mine = jira.search_issues('project=PROJ and assignee != currentUser()')

# my top 5 issues due by the end of the week, ordered by priority
oh_crap = jira.search_issues('assignee = currentUser() and due < endOfWeek() order by priority desc', maxResults=5)

# Summaries of my last 3 reported issues
for issue in jira.search_issues('reporter = currentUser() order by created desc', maxResults=3):
    print('{}: {}'.format(issue.key, issue.fields.summary))

JIRAError: JiraError HTTP 400 url: https://plutotv.atlassian.net/rest/api/2/search?jql=project%3DPROJ&startAt=0&validateQuery=True&maxResults=50
	text: The value 'PROJ' does not exist for the field 'project'.
	
	response headers = {'Server': 'AtlassianProxy/1.15.8.1', 'Cache-Control': 'no-cache, no-store, no-transform', 'Content-Type': 'application/json;charset=UTF-8', 'Strict-Transport-Security': 'max-age=315360000; includeSubDomains; preload', 'Date': 'Thu, 29 Oct 2020 17:08:18 GMT', 'ATL-TraceId': '3222df3413758adb', 'X-AREQUESTID': '3db06c60-3824-42af-9a94-e17df6e87c81', 'X-XSS-Protection': '1; mode=block', 'Transfer-Encoding': 'chunked', 'Timing-Allow-Origin': '*', 'X-Content-Type-Options': 'nosniff', 'Connection': 'close', 'Expect-CT': 'report-uri="https://web-security-reports.services.atlassian.com/expect-ct-report/global-proxy", enforce, max-age=86400'}
	response text = {"errorMessages":["The value 'PROJ' does not exist for the field 'project'."],"warningMessages":[]}

## Projects

In [9]:
projects = jira.projects()
jra = jira.project('JRA')
print(jra.name)                 # 'JIRA'
print(jra.lead.displayName)     # 'John Doe [ACME Inc.]'

JIRAError: JiraError HTTP 404 url: https://plutotv.atlassian.net/rest/api/2/project/JRA
	text: No project could be found with key 'JRA'.
	
	response headers = {'Server': 'AtlassianProxy/1.15.8.1', 'Vary': 'Accept-Encoding', 'Cache-Control': 'no-cache, no-store, no-transform', 'Content-Type': 'application/json;charset=UTF-8', 'Content-Encoding': 'gzip', 'Strict-Transport-Security': 'max-age=315360000; includeSubDomains; preload', 'Date': 'Thu, 29 Oct 2020 17:08:21 GMT', 'ATL-TraceId': '09ca6e70f60e153c', 'X-AREQUESTID': '2e90050e-11da-4745-bd8c-a08c84b5194d', 'X-XSS-Protection': '1; mode=block', 'Transfer-Encoding': 'chunked', 'Timing-Allow-Origin': '*', 'X-Content-Type-Options': 'nosniff', 'Connection': 'keep-alive', 'Expect-CT': 'report-uri="https://web-security-reports.services.atlassian.com/expect-ct-report/global-proxy", enforce, max-age=86400'}
	response text = {"errorMessages":["No project could be found with key 'JRA'."],"errors":{}}

In [10]:
##THESE ARE ALL UNDER THE JIRA OBJECT
#boards
#issue
#sprint
#sprint_info
#sprints_by_name
#statuscategories()
#user

In [ ]:
#Method to establish a new client connection

def create_jira_client(user, token, instance_name):
    options = {'server': 'https://{}.atlassian.net'.format(instance_name)}
    return JIRA(options, basic_auth=(user, token))

#Method to access lead time

def get_lead_time(jira: JIRA, project: str, dt: str):
    query = 'project={} and created > {} and status=Done and type=Story'
    issues = [jira.issue(x) for x in jira.search_issues(query.format(project, dt))]
    return [[x.key,  x.fields.summary, 'Story', x.fields.created, x.fields.updated] for x in issues]

#Method to obtain number of incidences

def get_incidents(jira: JIRA, project: str, dt: str):
    query = 'project={} and created > {} and type=Bug'
    issues = [jira.issue(x) for x in jira.search_issues(query.format(project, dt))]
    return [[x.key,  x.fields.summary, 'Bug', x.fields.created, x.fields.updated] for x in issues]